In [6]:
# pycolmap,open3d做可视化，不做密集匹配（nocuda） colmap脚本程序可做密集匹配
import os
import pycolmap
import open3d

In [7]:
def SFM_re(image_dir,output_dir):
    os.makedirs(output_dir,exist_ok=True)
    database_path = os.path.join(output_dir, "database.db")
    # 特征提取
    pycolmap.extract_features(database_path, image_dir)
    # 特征匹配
    pycolmap.match_exhaustive(database_path)
    # 稀疏重建
    maps = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=image_dir,
            output_path=output_dir,
        )
    return maps

In [8]:
def visual_re(maps,output_dir):
    if not maps:
        print("No maps found")
        return

    reconstruction = list(maps.values())[0]
    # 提取点云
    points = []
    colors = []
    for point3D in reconstruction.points3D.values():
        points.append([point3D.xyz[0], point3D.xyz[1], point3D.xyz[2]])
        # 颜色需要归一化到0-1范围
        colors.append([point3D.color[0] / 255.0,
            point3D.color[1] / 255.0,
            point3D.color[2] / 255.0])
    # 创建点云
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    pcd.colors = open3d.utility.Vector3dVector(colors)
    # 保存点云结果
    ply_path = os.path.join(output_dir, "reconstruction.ply")
    open3d.io.write_point_cloud(ply_path, pcd)
    # 可视化点云
    open3d.visualization.draw_geometries([pcd],
                                     window_name="三维重建结果")

In [9]:
image="re-photo"
output="output"
result = SFM_re(image, output)
visual_re(result,output)

![point_cloud](point_cloud_result.jpg)